# Zcash Observatory data

Mitchell Krawiec-Thayer and Pranav Thirunavukkarasu

April 2020

Observatory R & D at [Insight](https://www.insightconsensus.com) supported by the [Zcash Foundation](https://grants.zfnd.org/proposals/21786689)

## Analysis Parameters

In [1]:
path_to_block_data = 'blocks_20200408.csv'
output_filename = 'augmented_data.csv'
block_time_s = 75
sync_cutoff = 10000 #ms
q_save_output = 1

## Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import data

In [3]:
df = pd.read_csv('blocks_20200408.csv') # Import to data frame
df.describe() # High level summary of the data set

,block_height,miner_time,valid_time
count,10281.000000,1.028100e+04,1.028100e+04
mean,781900.051649,1.585770e+09,1.585770e+12
std,3434.295589,2.587229e+05,2.587500e+08
min,776372.000000,1.585354e+09,1.585354e+12
25%,778941.000000,1.585547e+09,1.585547e+12
50%,781506.000000,1.585741e+09,1.585741e+12
75%,785262.000000,1.586023e+09,1.586024e+12
max,787835.000000,1.586218e+09,1.586219e+12


In [ ]:
df.head() # Take a peeke to make sure it imported correctly

,block_height,block_hash,prev_hash,miner_target,miner_time,valid_time,transaction_hashes,inv_timestamps
0,777350.0,00000000014a976f53c53c63a2d52c0246e4c9ecf260b9...,0000000001afacebda04eb4ef20b46b34715d042473409...,0000000002ffff00000000000000000000000000000000...,1.585428e+09,1.585428e+12,269090c8fa46f72c3090803a7ec5231d3e3f73c0035f3d...,1585427780187;1585427780195;1585427780309;1585...
1,776540.0,00000000016dbb649d01642fb8603d9cfb3337a38c3c36...,00000000008855524979caa571b3fe7031d3e769eb6190...,0000000002fc4d00000000000000000000000000000000...,1.585366e+09,1.585366e+12,bd093b58b615184b82c4d7d4c1395cf0e6477470830fa3...,1585366448977;1585366448944;1585366449144;1585...
2,777066.0,00000000007826f5d75ea2268aa6703ff3087a1419daa7...,00000000007b4d4a898c18c291243e39998e52bcf67795...,0000000002e0d600000000000000000000000000000000...,1.585406e+09,1.585406e+12,09821279cf3b13cac66fe4b6e57f3778f4621cee083a02...,1585406005599;1585406005554;1585406005524;1585...
3,777264.0,00000000031fac7a7d902a87dc887375b83fd64466304a...,00000000002fc315a0c67eb4b1c26e7c8b6f5055e3f1cf...,00000000033fdd00000000000000000000000000000000...,1.585422e+09,1.585422e+12,0e61aaff225ead8a7d175853a2a9c161e1765bae525bbf...,1585421660806;1585421660865;1585421660793;1585...
4,777150.0,0000000002a6698519dc8abf12e82d2d74e4d4518243d4...,0000000000bb617c3c9603ba991dce9feb5df9265697ed...,0000000002c29f00000000000000000000000000000000...,1.585412e+09,1.585412e+12,a08776d8cdeafbc4f675fcaf5a9dd73bc509249fff6d25...,1585412301058;1585412301031;1585412301126;1585...


## Feature Engineering

Add a handful of calculated columns

In [ ]:
fork_heights = set()
dupe_count_buffer = []
apparent_latency_buffer = []
peer_count_buffer = []
raw_prop_buffer = []
prop_buffer = []
txn_count_buffer = []
bounded_times_list = []
first_heard_list = []

for r in range(len(df)):
    
    # Grab height
    this_height = df.iloc[r].block_height
    
    # How many versions were received at this_height?
    this_count = len(df[df['block_height'] == this_height])
    dupe_count_buffer.append(this_count)
    if this_count > 1:
        fork_heights.add(int(this_height)) # note this makes duplicates, so remember to unique() later

    raw_timestamps = df.iloc[r].inv_timestamps
    # If multiple peers, inv_timestamps is parsed as a string, and must be split into list of integers
    if isinstance(raw_timestamps, str):
        peer_timestamps = [int(i) for i in raw_timestamps.split(';')] # split up the timestamps
        peer_count_buffer.append(int(len(peer_timestamps))) # record peer count
        apparent_latency_buffer.append(int(min(peer_timestamps) - 1000*df.iloc[r].miner_time))
        raw_prop_time = np.max(peer_timestamps)-np.min(peer_timestamps)
        raw_prop_buffer.append(raw_prop_time)
        first_heard_list.append(int(np.min(peer_timestamps)))
        if raw_prop_time < sync_cutoff:
            prop_buffer.append(raw_prop_time)
            bounded_times = peer_timestamps
        else:
            # Throw out the high timestamp and try again. This a temporary fix
            peer_offset_removed = [x - min(peer_timestamps) for x in peer_timestamps]
            bounded_times = [p for p in peer_offset_removed if p < sync_cutoff]
            prop_buffer.append(np.max(bounded_times) - np.min(bounded_times))
    else:
        if np.isnan(raw_timestamps):
            # Did not catch this live from any peers
            peer_timestamps = np.nan 
            peer_count_buffer.append(np.nan)
            raw_prop_buffer.append(np.nan)
            prop_buffer.append(np.nan)
            apparent_latency_buffer.append(np.nan)
            bounded_times = (np.nan)
            first_heard_list.append(np.nan)
        else:
            # Caught one copy, parsed as a float
            peer_timestamps = int(raw_timestamps)
            peer_count_buffer.append(1)
            raw_prop_buffer.append(np.nan)
            prop_buffer.append(np.nan)
            apparent_latency_buffer.append(peer_timestamps - 1000*df.iloc[r].miner_time)
            bounded_times = (peer_timestamps)
            first_heard_list.append(peer_timestamps)
       
    bounded_times_list.append(bounded_times)
        
    txn_list = str(df.iloc[r].transaction_hashes)
    if isinstance(txn_list, str):
        txn_count_buffer.append(len(txn_list.split(';')))
    else:
        txn_count_buffer.append(1)
        
df['dupe_count'] = dupe_count_buffer
df['apparent_latency_ms'] = apparent_latency_buffer
df['peer_count'] = peer_count_buffer
df['raw_prop_ms'] = raw_prop_buffer
df['prop_envelope_ms'] = prop_buffer
df['txn_count'] = txn_count_buffer
df['bounded_p2p_times'] = bounded_times_list
df['first_heard'] = first_heard_list

Calculate block differences based on the **miner-reported timestamp**. Remember that these can and often are spoofed.

In [ ]:
# Calculate block times
miner_time_diff_buffer = []

for r in range(len(df)):

    this_height = df.iloc[r].block_height
    prev_indices = df.index[df['block_height'] == this_height-1].tolist()

    if len(prev_indices) == 0: 
        # If we didn't record previous block, then NaN and move on
        miner_time_diff_buffer.append(np.nan)
    else:
        # Get the index of (any one version of) the previous block
        if len(prev_indices) == 1:
            use_index = prev_indices
        else:
            use_index = int(min(prev_indices)) # pick only one index
            
        last_miner_time = int(df.iloc[use_index].miner_time) # miner time from last block
        this_miner_time = df.iloc[r].miner_time # miner time from this block
        miner_time_diff = (this_miner_time - last_miner_time) # difference
        miner_time_diff_buffer.append(miner_time_diff) # record

df['miner_reported_time_diff'] = miner_time_diff_buffer

Now calculate the **REAL** block time based on observatory timestamps ("bounded_p2p_timestamps" column!)

In [ ]:
# Calculate block times
real_block_time_buffer = []
bounded_times_list = []

for r in range(len(df)):

    this_height = df.iloc[r].block_height
    prev_indices = df.index[df['block_height'] == this_height-1].tolist()

    if len(prev_indices) == 0: 
        # If we didn't record previous block, then NaN and move on
        real_block_time_buffer.append(np.nan)
    else:
        # Get the index of (any one version of) the previous block
        if len(prev_indices) == 1:
            use_index = prev_indices
        else:
            use_index = int(min(prev_indices)) # pick only one index
            
        last_heard_raw = df.iloc[use_index].first_heard 
        this_heard_raw = df.iloc[r].first_heard 
        try:
            last_real_time = int(pd.to_numeric(last_heard_raw))
        except:
            last_real_time = np.nan
        try:
            this_real_time = int(pd.to_numeric(this_heard_raw))
        except:
            this_real_time = np.nan
        real_time_diff = (this_real_time - last_real_time)/1000
        real_block_time_buffer.append(real_time_diff)
    
df['real_block_time'] = real_block_time_buffer

In [ ]:
df.sort_values(by='block_height').head()

In [ ]:
if q_save_output:
    df.to_csv(output_filename, index=False)

## Visualizations

### When did we encounter forks?

In [ ]:
uptime_blocks = np.count_nonzero(~np.isnan(df.peer_count))
print('Encountered ' + str(len(fork_heights)) + ' forks, during ' + str(uptime_blocks) + ' blocks of operation')
print('                             (' + str(round(uptime_blocks*block_time_s/60/60)) + ' hours)')
print('                             (' + str(round(uptime_blocks*block_time_s/60/60/24)) + ' days)')
print('This represents a ' + str(round(len(fork_heights)/uptime_blocks*100,2)) + '% fork rate')

height_hist = plt.figure(figsize=(8,5), facecolor='w')
plt.hist(list(fork_heights));
plt.xlabel('Time (height)')
plt.ylabel('Fork count')
plt.title('Fork detection by Zcash observatory (insight-node-0)');

## Comparing miner timestamps against actual receipt
Please note that the miner-reported timestamp is a semi-arbitrary value selected by the miner when they initially generate the block headers. You should not assume that ANY of them are correct.

In [ ]:
height_hist = plt.figure(figsize=(8,5), facecolor='w')
plt.hist(df.apparent_latency_ms/1000, bins=np.logspace(np.log10(1),np.log10(1e5), 50));
plt.yscale('log')
plt.xscale('log')
plt.xlabel('*apparent* latency (sec)')
plt.ylabel('Frequency')
plt.title('Miner timestamp - observatory receipt timestamp');

## Propagation time

In [ ]:
height_hist = plt.figure(figsize=(10,5), facecolor='w')
plt.hist(df['prop_envelope_ms'], bins=np.logspace(np.log10(min(df['prop_envelope_ms'][df['prop_envelope_ms']>0])),np.log10(max(df['prop_envelope_ms'])), 100) );
plt.yscale('log')
plt.xscale('log')
plt.xlabel('last-first timestamp (ms)')
plt.ylabel('Frequency')
plt.title('Observed block propagation time (insight-node-0)')

In [ ]:
# How does the observed propagation envelope depend on the number of transactions in a block
peer_prop_scatter = plt.figure(figsize=(8,5), facecolor='w')
plt.scatter(df['txn_count'], df['prop_envelope_ms'])
plt.xlabel('Transactions in block')
plt.ylabel('Observed propagation time (ms)')
plt.yscale('linear')
plt.xscale('linear')
plt.title('Impact of block size on propagation');

In [ ]:
# How does the observed propagation envelope depend on the number of peers that we hear the block from
peer_prop_scatter = plt.figure(figsize=(8,5), facecolor='w')
plt.scatter(df['peer_count'], df['prop_envelope_ms'])
plt.xlabel('Peer count')
plt.ylabel('Observed propagation time (ms)')
plt.title('Peer count <-> observed prop time');

Note that that we can see how fast remote nodes are syncing, based on when they ask for blocks

In [ ]:
peer_prop_scatter = plt.figure(figsize=(15,5), facecolor='w')
plt.scatter(df['block_height'], df['raw_prop_ms'])
plt.xlabel('Time (height)')
plt.ylabel('Observed propagation time (ms)')
plt.title('');

## Block time

In [ ]:
height_hist = plt.figure(figsize=(10,5), facecolor='w')
plt.hist(df['real_block_time'], bins=np.linspace(0,500,100))
plt.xlabel('Observed block time (s)')
plt.ylabel('Frequency')
plt.title('Observed block time [insight-node-0] \n (observatory peer-receipt timestamps, NOT miner-reported timestamps)');

In [ ]:
height_hist = plt.figure(figsize=(10,5), facecolor='w')
plt.hist(df['real_block_time'], bins=np.linspace(0,2,40))
plt.xlabel('Observed block time (s)')
plt.ylabel('Frequency')
plt.ylabel('Frequency')
plt.title('Observed block time [insight-node-0] \n (observatory peer-receipt timestamps, NOT miner-reported timestamps)');

In [ ]:
# Comparing block time from miner timestamps to real timestamps
peer_prop_scatter = plt.figure(figsize=(8,5), facecolor='w')
plt.scatter(df['miner_reported_time_diff'], df['real_block_time'])
plt.xlabel('Miner reported block time')
plt.ylabel('Observed block time')
plt.title('');
plt.ylim((0,800));